In [ ]:
!pip install datasets -q
!pip install transformers -q
!pip uninstall -y wandb -q

In [ ]:
import csv
import pandas as pd
from datasets import Dataset
from datasets import load_from_disk
from transformers import BertTokenizer, DataCollatorForLanguageModeling, BertForMaskedLM, Trainer, TrainingArguments

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dataset = pd.read_csv('/content/train_dataset.csv', encoding='utf-8', on_bad_lines='skip') #quoting=csv.QUOTE_NONE
train_dataset

In [ ]:
train_dataset = train_dataset.drop(columns=['Unnamed: 0'])
train_dataset = train_dataset.dropna(subset=['polypers'])
train_dataset = train_dataset.reset_index(drop=True)
train_dataset

,base,polypers,was_changed
0,« Опасный беглец » .,« Опасный беглец » .,False
1,— Эта книга рассказывает о народной освободите...,— Эта книга рассказывает о народной освободите...,False
2,В середине прошлого века угнетенные народы Инд...,В середине прошлого века угнетенные народы Инд...,False
3,Колонизаторы душили всякое проявление свободно...,Колонизаторы душилиет всякое проявление свобод...,True
4,Свидетелем страшной расправы голландских колон...,Свидетелем страшной расправы голландских колон...,False
...,...,...,...
1540834,Осадчий же сейчас повел себя не лучше обросимо...,Осадчий же сейчас повелет себя не лучше оброси...,True
1540835,— Парадокс с синтезом твоим .,— Парадокс с синтезом твоим .,False
1540836,"Чем дешевле все получится , тем печальнее …","Чем дешевле все получится , тем печальнее …",False
1540837,И он уже не смотрит на Зберовского .,И он уже не смотрит на Зберовского .,False


In [ ]:
dataset = Dataset.from_pandas(train_dataset[['polypers']])

In [ ]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['polypers'], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
save_path = '/content/drive/MyDrive/Магистратура/Пробинг/tokenized_datasets'

tokenized_datasets.save_to_disk(save_path)

In [ ]:
save_path = '/content/drive/MyDrive/Магистратура/Пробинг/tokenized_datasets'

tokenized_datasets = load_from_disk(save_path)

In [ ]:
tokenized_datasets = tokenized_datasets.select(range(2000))

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
train_dataset, test_dataset = tokenized_datasets.train_test_split(test_size=0.1).values()

In [ ]:
model = BertForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_steps=1000,
    save_total_limit=2,
    report_to=None,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-30-24df24c09e97>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
save_path = '/content/drive/MyDrive/Магистратура/Пробинг/rubert-finetuned'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/Магистратура/Пробинг/rubert-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/Магистратура/Пробинг/rubert-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/Магистратура/Пробинг/rubert-finetuned/vocab.txt',
 '/content/drive/MyDrive/Магистратура/Пробинг/rubert-finetuned/added_tokens.json')